## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("./Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis = 1)


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#Taking cutoff of 100 her
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts() < 100
application_types_to_replace = application_types_to_replace[application_types_to_replace].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

#Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1570        1
C6100        1
C1728        1
C2170        1
C1900        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C2400        6
C1720        6
C4100        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1246        2
C1234        2
C3200        2
C1256        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts() <= 300
classifications_to_replace = classifications_to_replace[classifications_to_replace].index.tolist()

# Replace in dataframe
for classif in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(classif,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
encoded_df = pd.get_dummies(application_df, columns = ['CLASSIFICATION','INCOME_AMT','SPECIAL_CONSIDERATIONS','APPLICATION_TYPE','AFFILIATION','USE_CASE',"ORGANIZATION"])
encoded_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,CLASSIFICATION_C1000,CLASSIFICATION_C1200,CLASSIFICATION_C2000,CLASSIFICATION_C2100,CLASSIFICATION_C3000,CLASSIFICATION_C7000,CLASSIFICATION_Other,...,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,1,142590,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [10]:
# Split our preprocessed data into our features and target arrays, then train/test
X_train, X_test, y_train, y_test = train_test_split(encoded_df.drop(["IS_SUCCESSFUL"], axis=1 ), 
                                                    encoded_df["IS_SUCCESSFUL"], 
                                                    test_size = 0.7, 
                                                    random_state = 0)



In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
X_train_scaled.shape

(10289, 45)

## Compile, Train and Evaluate the Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=24, activation="relu", input_dim=45))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=22))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim=8))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 24)                1104      
                                                                 
 dense_1 (Dense)             (None, 8)                 200       
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 1,313
Trainable params: 1,313
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
checkpoint_path = "./training/cp.ckpt"

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1,
                                                save_freq = 'epoch',
                                                period = 5)

fit_model = nn.fit(X_train_scaled, y_train, epochs=100, validation_data = (X_test, y_test), callbacks = cp_callback)

Epoch 1/100
322/322 [==============================] - 1s 2ms/step - loss: 0.6029 - accuracy: 0.7104 - val_loss: 105656.3047 - val_accuracy: 0.4681
Epoch 2/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5580 - accuracy: 0.7276 - val_loss: 78689.2578 - val_accuracy: 0.4681
Epoch 3/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5501 - accuracy: 0.7347 - val_loss: 62482.0508 - val_accuracy: 0.4681
Epoch 4/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5456 - accuracy: 0.7371 - val_loss: 78800.5469 - val_accuracy: 0.4681
Epoch 5/100
309/322 [===========================>..] - ETA: 0s - loss: 0.5420 - accuracy: 0.7394
Epoch 5: saving model to ./training\cp.ckpt
322/322 [==============================] - 1s 2ms/step - loss: 0.5431 - accuracy: 0.7386 - val_loss: 55274.8047 - val_accuracy: 0.4681
Epoch 6/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5411 - accuracy: 0.7377 - val_loss: 63456.5234 - val_accur

322/322 [==============================] - 1s 2ms/step - loss: 0.5267 - accuracy: 0.7456 - val_loss: 41511.5742 - val_accuracy: 0.4681
Epoch 48/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5252 - accuracy: 0.7467 - val_loss: 49095.9609 - val_accuracy: 0.4681
Epoch 49/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5257 - accuracy: 0.7460 - val_loss: 39692.1797 - val_accuracy: 0.4681
Epoch 50/100
317/322 [============================>.] - ETA: 0s - loss: 0.5260 - accuracy: 0.7470
Epoch 50: saving model to ./training\cp.ckpt
322/322 [==============================] - 1s 2ms/step - loss: 0.5256 - accuracy: 0.7474 - val_loss: 46342.1094 - val_accuracy: 0.4681
Epoch 51/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5254 - accuracy: 0.7472 - val_loss: 42990.6406 - val_accuracy: 0.4681
Epoch 52/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5252 - accuracy: 0.7463 - val_loss: 36846.2461 - val_accuracy: 0.

322/322 [==============================] - 1s 2ms/step - loss: 0.5212 - accuracy: 0.7489 - val_loss: 73240.2578 - val_accuracy: 0.4681
Epoch 95/100
316/322 [============================>.] - ETA: 0s - loss: 0.5218 - accuracy: 0.7494
Epoch 95: saving model to ./training\cp.ckpt
322/322 [==============================] - 1s 2ms/step - loss: 0.5211 - accuracy: 0.7499 - val_loss: 80886.4766 - val_accuracy: 0.4681
Epoch 96/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5210 - accuracy: 0.7511 - val_loss: 74473.4375 - val_accuracy: 0.4681
Epoch 97/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5216 - accuracy: 0.7503 - val_loss: 71908.1328 - val_accuracy: 0.4681
Epoch 98/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5207 - accuracy: 0.7502 - val_loss: 57902.4141 - val_accuracy: 0.4681
Epoch 99/100
322/322 [==============================] - 1s 2ms/step - loss: 0.5211 - accuracy: 0.7492 - val_loss: 64807.6680 - val_accuracy: 0.

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

751/751 - 0s - loss: 0.5705 - accuracy: 0.7237 - 438ms/epoch - 583us/step
Loss: 0.570456862449646, Accuracy: 0.7237401008605957


In [20]:
# Export our model to HDF5 file
nn.save(filepath = "model.h5")